In [2]:
# TODO change scaling mechanism to work with EPSG:4326 with approximated meters

import concurrent.futures
import os

import numpy as np
import rasterio
import tqdm

In [ ]:
images_dir = "/net/data_ssd/tree_mortality_orthophotos/orthophotos"
gw_path = "/net/home/jmoehring/gdal-3.7.3/build/bin/gdal_warp"
jobs = 50

In [3]:
# add helper method to shorten cell size list
def shorten_list(arr, target):
    # Find the index of the first number greater than the target
    index = next((i for i, x in enumerate(arr) if x > target), None)

    if index is not None:
        # Shorten the list from the index onward
        shortened_list = arr[index:]
        return shortened_list
    else:
        # If no bigger number is found, return an empty array
        return []


def process_image(image_path, image_cell_width, gw_path):
    image_cell_width = round(image_cell_width, 2)
    scaled_image_path = image_path.replace(
        "_8857", "_8857_{0:.2f}".format(image_cell_width)
    )
    os.system(
        f"{gw_path} -q -tr {image_cell_width} {image_cell_width} {image_path} {scaled_image_path}"
    )

In [ ]:
def process_file(filename, args, cell_widths):
    filepath = os.path.join(args.images_dir, filename)
    if filename.endswith("8857.tif") or filename.endswith("8857_mask.tif"):
        with rasterio.open(filepath) as image:
            # find desired cell widths for the given image
            image_cell_widths = shorten_list(cell_widths, image.transform[0])

            with concurrent.futures.ProcessPoolExecutor(
                max_workers=args.jobs
            ) as executor:
                # Use ThreadPoolExecutor for parallel processing
                futures = [
                    executor.submit(process_image, filepath, cw, args.gw_path)
                    for cw in image_cell_widths
                ]
                # Wait for all futures to complete
                concurrent.futures.wait(futures)